In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True, transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=2)
testset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import os
import math
from abc import abstractmethod

from PIL import Image
import requests
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# beta schedule
def linear_beta_schedule(timesteps):
    scale = 1000 / timesteps
    beta_start = scale * 0.0001
    beta_end = scale * 0.02
    return torch.linspace(beta_start, beta_end, timesteps, dtype=torch.float64)

def cosine_beta_schedule(timesteps, s=0.008):
    """
    cosine schedule
    as proposed in https://arxiv.org/abs/2102.09672
    """
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps, dtype=torch.float64)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0, 0.999)

In [ ]:
class GaussianDiffusion:
    def __init__(
        self,
        timesteps=1000,
        beta_schedule='linear',
        #the label of the image we want to generate
        #label
        ):
        self.timesteps = timesteps

        if beta_schedule == 'linear':
            betas = linear_beta_schedule(timesteps)
        elif beta_schedule == 'cosine':
            betas = cosine_beta_schedule(timesteps)
        else:
            raise ValueError(f'unknown beta schedule {beta_schedule}')
        self.betas = betas

        self.alphas = 1. - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, axis=0)
        self.alphas_cumprod_prev = F.pad(self.alphas_cumprod[:-1], (1, 0), value=1.)

        # calculations for diffusion q(x_t | x_{t-1}) and others
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - self.alphas_cumprod)
        self.log_one_minus_alphas_cumprod = torch.log(1.0 - self.alphas_cumprod)
        self.sqrt_recip_alphas_cumprod = torch.sqrt(1.0 / self.alphas_cumprod)
        self.sqrt_recipm1_alphas_cumprod = torch.sqrt(1.0 / self.alphas_cumprod - 1)

        # calculations for posterior q(x_{t-1} | x_t, x_0)
        self.posterior_variance = (
            self.betas * (1.0 - self.alphas_cumprod_prev) / (1.0 - self.alphas_cumprod)
        )
        # below: log calculation clipped because the posterior variance is 0 at the beginning
        # of the diffusion chain
        self.posterior_log_variance_clipped = torch.log(self.posterior_variance.clamp(min =1e-20))

        self.posterior_mean_coef1 = (
            self.betas * torch.sqrt(self.alphas_cumprod_prev) / (1.0 - self.alphas_cumprod)
        )
        self.posterior_mean_coef2 = (
            (1.0 - self.alphas_cumprod_prev)
            * torch.sqrt(self.alphas)
            / (1.0 - self.alphas_cumprod)
        )

        #suppose we want to generate the image with label y
        #self.label=label



    # get the param of given timestep t
    def _extract(self, a, t, x_shape):
        batch_size = t.shape[0]
        out = a.to(t.device).gather(0, t).float()
        out = out.reshape(batch_size, *((1,) * (len(x_shape) - 1)))
        return out

    # forward diffusion (using the nice property): q(x_t | x_0)
    def q_sample(self, x_start, t, noise=None):
        if noise is None:
            noise = torch.randn_like(x_start)

        sqrt_alphas_cumprod_t = self._extract(self.sqrt_alphas_cumprod, t, x_start.shape)
        sqrt_one_minus_alphas_cumprod_t = self._extract(self.sqrt_one_minus_alphas_cumprod, t, x_start.shape)

        return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

    # Get the mean and variance of q(x_t | x_0).
    def q_mean_variance(self, x_start, t):
        mean = self._extract(self.sqrt_alphas_cumprod, t, x_start.shape) * x_start
        variance = self._extract(1.0 - self.alphas_cumprod, t, x_start.shape)
        log_variance = self._extract(self.log_one_minus_alphas_cumprod, t, x_start.shape)
        return mean, variance, log_variance

    # Compute the mean and variance of the diffusion posterior: q(x_{t-1} | x_t, x_0)
    def q_posterior_mean_variance(self, x_start, x_t, t):
        posterior_mean = (
            self._extract(self.posterior_mean_coef1, t, x_t.shape) * x_start
            + self._extract(self.posterior_mean_coef2, t, x_t.shape) * x_t
        )
        posterior_variance = self._extract(self.posterior_variance, t, x_t.shape)
        posterior_log_variance_clipped = self._extract(self.posterior_log_variance_clipped, t, x_t.shape)
        return posterior_mean, posterior_variance, posterior_log_variance_clipped

    # compute x_0 from x_t and pred noise: the reverse of `q_sample`
    def predict_start_from_noise(self, x_t, t, noise):
        return (
            self._extract(self.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t -
            self._extract(self.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise
        )


    #get the gradient of the classifier on the input x
    def get_gradient(self,x_t,x,y):
        with torch.enable_grad():
          x_t_in=x.detach().requires_grad_(True)
          label_tensor = classifier(x_t)
          selected=label_tensor[y]

          return torch.autograd.grad(y,x_t_in)[0]*arg.classifier_scale


    # compute predicted mean and variance of p(x_{t-1} | x_t)
    def p_mean_variance(self, model, x_t, t, clip_denoised=True):
        # predict noise using model
        pred_noise = model(x_t, t)
        # get the predicted x_0: different from the algorithm2 in the paper
        x_recon = self.predict_start_from_noise(x_t, t, pred_noise)
        if clip_denoised:
            x_recon = torch.clamp(x_recon, min=-1., max=1.)
        model_mean, posterior_variance, posterior_log_variance = \
                    self.q_posterior_mean_variance(x_recon, x_t, t)


        #we add the gradient to the model mean to direct the model to generate the image to the direction we want

        gradient=self.get_gradient(x_t,x,y)

        model_mean+=model+posterior_variance*gradient

        return model_mean, posterior_variance, posterior_log_variance

    # denoise_step: sample x_{t-1} from x_t and pred_noise
    @torch.no_grad()
    def p_sample(self, model, x_t, t, clip_denoised=True):
        # predict mean and variance
        model_mean, _, model_log_variance = self.p_mean_variance(model, x_t, t,
                                                    clip_denoised=clip_denoised)
        noise = torch.randn_like(x_t)
        # no noise when t == 0
        nonzero_mask = ((t != 0).float().view(-1, *([1] * (len(x_t.shape) - 1))))
        # compute x_{t-1}
        pred_img = model_mean + nonzero_mask * (0.5 * model_log_variance).exp() * noise
        return pred_img

    # denoise: reverse diffusion
    @torch.no_grad()
    def p_sample_loop(self, model, shape):
        batch_size = shape[0]
        device = next(model.parameters()).device
        # start from pure noise (for each example in the batch)
        img = torch.randn(shape, device=device)
        imgs = []
        for i in tqdm(reversed(range(0, timesteps)), desc='sampling loop time step', total=timesteps):
            img = self.p_sample(model, img, torch.full((batch_size,), i, device=device, dtype=torch.long))
            imgs.append(img.cpu().numpy())
        return imgs

    # sample new images
    @torch.no_grad()
    def sample(self, model, image_size, batch_size=8, channels=3):
        return self.p_sample_loop(model, shape=(batch_size, channels, image_size, image_size))

    # compute train losses
    def train_losses(self, model, x_start, t):
        # generate random noise
        noise = torch.randn_like(x_start)
        # get x_t
        x_noisy = self.q_sample(x_start, t, noise=noise)
        predicted_noise = model(x_noisy, t)
        loss = F.mse_loss(noise, predicted_noise)
        return loss

In [ ]:
import torch
from torchvision import datasets, transforms
import numpy as np
import pandas as pd

# Download and prepare CIFAR-10 dataset
image_size = 128
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

cifar10_data = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
data_loader = torch.utils.data.DataLoader(cifar10_data, batch_size=128, shuffle=True)

gaussian_diffusion = GaussianDiffusion(timesteps=1000)

Files already downloaded and verified


In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision import transforms
train_dataset_full, test_dataset_full = train_test_split(cifar10_data, test_size=0.2, random_state=42)
#train_dataset_full = CustomTensorDataset(train_df_full)
#test_dataset_full = CustomTensorDataset(test_df_full)
train_loader_full = DataLoader(train_dataset_full, batch_size=32, shuffle=True)
test_loader_full = DataLoader(test_dataset_full, batch_size=32, shuffle=False)
train_loader_full_list=[train_loader_full]
test_loader_full_list=[test_loader_full]

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision import transforms
#create new dataloaders which contain 1000 images which are randomly chosen from cifar-10
full_dataset = data_loader.dataset

def get_subset_loader_1000(full_dataset,datasize):

  # 随机抽取1000个不重复的索引
  subset_indices = torch.randperm(len(full_dataset))[:datasize]

  # 创建一个数据子集
  subset = Subset(full_dataset, subset_indices)

  # 创建一个新的DataLoader，它只从子集中加载数据
  # 注意: 你可能想保持与原始DataLoader相同的batch_size和其他参数
  subset_dataloader = DataLoader(subset, batch_size=data_loader.batch_size, shuffle=True)

  return subset,subset_dataloader

In [ ]:
def get_clean_data(sub_data_loader):
    print('Function called.')
    results = []
    print('Initial results length:', len(results))

    for images, labels in sub_data_loader:
        for i in range(len(images)):
           #print(len(images))
            x_start = images[i]

            result = {
                'noisy': x_start,
                'label': labels[i].item()
            }
            results.append(result)
        print('Batch done. Current results length:', len(results))

    print('Final results length:', len(results))
    df = pd.DataFrame(results)
    return df

In [ ]:
#get clean train test loader
from sklearn.model_selection import train_test_split

dataset_10000,dataset_10000_loader=get_subset_loader_1000(cifar10_data,10000)
#train_df_clean, test_df_clean = train_test_split(dataset_10000, test_size=0.2, random_state=42)
clean_dataframe=get_clean_data(dataset_10000_loader)
train_df_clean, test_df_clean = train_test_split(clean_dataframe, test_size=0.2, random_state=42)

Function called.
Initial results length: 0
Batch done. Current results length: 128
Batch done. Current results length: 256
Batch done. Current results length: 384
Batch done. Current results length: 512
Batch done. Current results length: 640
Batch done. Current results length: 768
Batch done. Current results length: 896
Batch done. Current results length: 1024
Batch done. Current results length: 1152
Batch done. Current results length: 1280
Batch done. Current results length: 1408
Batch done. Current results length: 1536
Batch done. Current results length: 1664
Batch done. Current results length: 1792
Batch done. Current results length: 1920
Batch done. Current results length: 2048
Batch done. Current results length: 2176
Batch done. Current results length: 2304
Batch done. Current results length: 2432
Batch done. Current results length: 2560
Batch done. Current results length: 2688
Batch done. Current results length: 2816
Batch done. Current results length: 2944
Batch done. Current r

In [ ]:
train_dataset_clean = CustomTensorDataset(train_df_clean)
test_dataset_clean = CustomTensorDataset(test_df_clean)
train_loader_clean = DataLoader(train_dataset_clean , batch_size=32, shuffle=True)
test_loader_clean = DataLoader(test_dataset_clean, batch_size=32, shuffle=False)
train_loader_clean_lst=[train_loader_clean]
test_loader_clean_lst=[test_loader_clean]

NameError: ignored

In [ ]:
subset,_t=get_subset_loader_1000(full_dataset,5000)
print(subset)
print(f"Size of subset: {len(subset)}")
first_data_point = subset[0]
print(f"First data point (features, label): {first_data_point}")

Size of subset: 5000
First data point (features, label): (tensor([[[0.7804, 0.7804, 0.7804,  ..., 0.9294, 0.9294, 0.9294],
         [0.7804, 0.7804, 0.7804,  ..., 0.9294, 0.9294, 0.9294],
         [0.7804, 0.7804, 0.7804,  ..., 0.9137, 0.9137, 0.9137],
         ...,
         [0.3725, 0.3725, 0.3725,  ..., 0.1608, 0.1608, 0.1608],
         [0.3725, 0.3725, 0.3725,  ..., 0.1686, 0.1686, 0.1686],
         [0.3725, 0.3725, 0.3725,  ..., 0.1686, 0.1686, 0.1686]],

        [[0.7647, 0.7647, 0.7647,  ..., 0.9294, 0.9294, 0.9294],
         [0.7647, 0.7647, 0.7647,  ..., 0.9294, 0.9294, 0.9294],
         [0.7647, 0.7647, 0.7647,  ..., 0.9137, 0.9137, 0.9137],
         ...,
         [0.2941, 0.2941, 0.2941,  ..., 0.0902, 0.0902, 0.0902],
         [0.2941, 0.2941, 0.2941,  ..., 0.0980, 0.0980, 0.0980],
         [0.2941, 0.2941, 0.2941,  ..., 0.0980, 0.0980, 0.0980]],

        [[0.8588, 0.8588, 0.8588,  ..., 0.9294, 0.9294, 0.9294],
         [0.8588, 0.8588, 0.8588,  ..., 0.9294, 0.9294, 0.9294],


In [ ]:
#generate the date for training the classifier
#the ourput of the function contains 1000 data of the noisy image in the corresponding timestep interval. In which we take 1000 images randomly in cifar 10 dataset and add noise to a random step
#in the interval we set
import random

def get_training_data(begin, end, sub_data_loader):
    print('Function called.')
    results = []
    print('Initial results length:', len(results))

    for images, labels in sub_data_loader:
        for i in range(len(images)):
           #print(len(images))
            x_start = images[i]
            #print('this is x_start',x_start)
            t = random.randint(begin, end)
            #print('this is timestep',t)
            x_noisy = gaussian_diffusion.q_sample(x_start, t=torch.tensor([t]))

            result = {
                'noisy': x_noisy,
                'label': labels[i].item()
            }
            results.append(result)
        print('Batch done. Current results length:', len(results))

    print('Final results length:', len(results))
    df = pd.DataFrame(results)
    return df


In [ ]:
#get the training data for 10 different classifiers
folder_path = '/content/drive/My Drive/classifier_noised_training_data'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

#_,subset_1_loader=get_subset_loader_1000(full_dataset,5000)
#training_data_1=get_training_data(1,100,data_loader)
#training_data_1.to_csv(f'{folder_path}/training_data_1.csv', index=False)

#_,subset_2_loader=get_subset_loader_1000(full_dataset,5000)
#training_data_2=get_training_data(101,200,data_loader)
#training_data_2.to_csv(f'{folder_path}/training_data_1.csv', index=False)

#_,subset_3_loader=get_subset_loader_1000(full_dataset,5000)
training_data_3=get_training_data(201,300,data_loader)
'''
#_,subset_4_loader=get_subset_loader_1000(full_dataset,5000)
training_data_4=get_training_data(301,400,data_loader)

#_,subset_5_loader=get_subset_loader_1000(full_dataset,5000)
training_data_5=get_training_data(401,500,data_loader)

#_,subset_6_loader=get_subset_loader_1000(full_dataset,5000)
training_data_6=get_training_data(501,600,data_loader)

#_,subset_7_loader=get_subset_loader_1000(full_dataset,5000)
training_data_7=get_training_data(601,700,data_loader)

#_,subset_8_loader=get_subset_loader_1000(full_dataset,5000)
training_data_8=get_training_data(701,800,data_loader)

#_,subset_9_loader=get_subset_loader_1000(full_dataset,5000)
training_data_9=get_training_data(801,900,data_loader)

#_,subset_10_loader=get_subset_loader_1000(full_dataset,5000)
training_data_10=get_training_data(901,999,data_loader)'''

Function called.
Initial results length: 0
Batch done. Current results length: 128
Batch done. Current results length: 256
Batch done. Current results length: 384
Batch done. Current results length: 512
Batch done. Current results length: 640
Batch done. Current results length: 768
Batch done. Current results length: 896
Batch done. Current results length: 1024
Batch done. Current results length: 1152
Batch done. Current results length: 1280
Batch done. Current results length: 1408
Batch done. Current results length: 1536
Batch done. Current results length: 1664
Batch done. Current results length: 1792
Batch done. Current results length: 1920
Batch done. Current results length: 2048
Batch done. Current results length: 2176
Batch done. Current results length: 2304
Batch done. Current results length: 2432
Batch done. Current results length: 2560
Batch done. Current results length: 2688
Batch done. Current results length: 2816
Batch done. Current results length: 2944
Batch done. Current r

'\n#_,subset_3_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_3=get_training_data(201,300,data_loader)\n\n#_,subset_4_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_4=get_training_data(301,400,data_loader)\n\n#_,subset_5_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_5=get_training_data(401,500,data_loader)\n\n#_,subset_6_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_6=get_training_data(501,600,data_loader)\n\n#_,subset_7_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_7=get_training_data(601,700,data_loader)\n\n#_,subset_8_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_8=get_training_data(701,800,data_loader)\n\n#_,subset_9_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_9=get_training_data(801,900,data_loader)\n\n#_,subset_10_loader=get_subset_loader_1000(full_dataset,5000)\ntraining_data_10=get_training_data(901,999,data_loader)'

In [ ]:
#define the custom dataset class
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

class CustomTensorDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # 假设第一列是转换后的图像，第二列是标签
        image = self.dataframe.iloc[idx, 0]  # 这里得到的是一个图像张量
        label = self.dataframe.iloc[idx, 1]
        return image, label

In [ ]:
#split the train test dataset for 10 subsets
#train_df_1, test_df_1 = train_test_split(training_data_1, test_size=0.2, random_state=42)
train_df_2, test_df_2 = train_test_split(training_data_2, test_size=0.2, random_state=42)
'''train_df_3, test_df_3 = train_test_split(training_data_3, test_size=0.2, random_state=42)
train_df_4, test_df_4 = train_test_split(training_data_4, test_size=0.2, random_state=42)
train_df_5, test_df_5 = train_test_split(training_data_5, test_size=0.2, random_state=42)
train_df_6, test_df_6 = train_test_split(training_data_6, test_size=0.2, random_state=42)
train_df_7, test_df_7 = train_test_split(training_data_7, test_size=0.2, random_state=42)
train_df_8, test_df_8 = train_test_split(training_data_8, test_size=0.2, random_state=42)
train_df_9, test_df_9 = train_test_split(training_data_9, test_size=0.2, random_state=42)
train_df_10, test_df_10 = train_test_split(training_data_10, test_size=0.2, random_state=42)'''

#create train and test datasets
#train_dataset_1 = CustomTensorDataset(train_df_1)
#test_dataset_1 = CustomTensorDataset(test_df_1)

#train_dataset_2 = CustomTensorDataset(train_df_2)
test_dataset_2 = CustomTensorDataset(test_df_2)
'''
train_dataset_3 = CustomTensorDataset(train_df_3)
test_dataset_3 = CustomTensorDataset(test_df_3)

train_dataset_4 = CustomTensorDataset(train_df_4)
test_dataset_4 = CustomTensorDataset(test_df_4)

train_dataset_5 = CustomTensorDataset(train_df_5)
test_dataset_5 = CustomTensorDataset(test_df_5)

train_dataset_6 = CustomTensorDataset(train_df_6)
test_dataset_6 = CustomTensorDataset(test_df_6)

train_dataset_7 = CustomTensorDataset(train_df_7)
test_dataset_7 = CustomTensorDataset(test_df_7)

train_dataset_8 = CustomTensorDataset(train_df_8)
test_dataset_8 = CustomTensorDataset(test_df_8)

train_dataset_9 = CustomTensorDataset(train_df_9)
test_dataset_9 = CustomTensorDataset(test_df_9)

train_dataset_10 = CustomTensorDataset(train_df_10)
test_dataset_10 = CustomTensorDataset(test_df_10)'''

#create train and test dataloaders
#train_loader_1 = DataLoader(train_dataset_1, batch_size=32, shuffle=True)
#test_loader_1 = DataLoader(test_dataset_1, batch_size=32, shuffle=False)

train_loader_2 = DataLoader(train_dataset_2, batch_size=32, shuffle=True)
test_loader_2 = DataLoader(test_dataset_2, batch_size=32, shuffle=False)
'''
train_loader_3 = DataLoader(train_dataset_3, batch_size=32, shuffle=True)
test_loader_3 = DataLoader(test_dataset_3, batch_size=32, shuffle=False)

train_loader_4 = DataLoader(train_dataset_4, batch_size=32, shuffle=True)
test_loader_4 = DataLoader(test_dataset_4, batch_size=32, shuffle=False)

train_loader_5 = DataLoader(train_dataset_5, batch_size=32, shuffle=True)
test_loader_5 = DataLoader(test_dataset_5, batch_size=32, shuffle=False)

train_loader_6 = DataLoader(train_dataset_6, batch_size=32, shuffle=True)
test_loader_6 = DataLoader(test_dataset_6, batch_size=32, shuffle=False)

train_loader_7 = DataLoader(train_dataset_7, batch_size=32, shuffle=True)
test_loader_7 = DataLoader(test_dataset_7, batch_size=32, shuffle=False)

train_loader_8 = DataLoader(train_dataset_8, batch_size=32, shuffle=True)
test_loader_8 = DataLoader(test_dataset_8, batch_size=32, shuffle=False)

train_loader_9 = DataLoader(train_dataset_9, batch_size=32, shuffle=True)
test_loader_9 = DataLoader(test_dataset_9, batch_size=32, shuffle=False)

train_loader_10 = DataLoader(train_dataset_10, batch_size=32, shuffle=True)
test_loader_10 = DataLoader(test_dataset_10, batch_size=32, shuffle=False)'''

train_loader_lst=[train_loader_2]
test_loader_lst=[test_loader_2]


In [ ]:
import torch.optim as optim
import torchvision.models as models
#create 10 different classifier models
classifier1=models.resnet50(pretrained=True)
classifier1.fc=nn.Sequential(
    nn.Linear(in_features=2048,out_features=1000,bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1000,out_features=10,bias=True),
    nn.LogSoftmax(dim=1)
)
classifier2=models.resnet50(pretrained=True)
classifier2.fc=nn.Sequential(
    nn.Linear(in_features=2048,out_features=1000,bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1000,out_features=10,bias=True),
    #nn.LogSoftmax(dim=1)
)
'''classifier3=models.resnet50(pretrained=True)
classifier4=models.resnet50(pretrained=True)
classifier5=models.resnet50(pretrained=True)
classifier6=models.resnet50(pretrained=True)
classifier7=models.resnet50(pretrained=True)
classifier8=models.resnet50(pretrained=True)
classifier9=models.resnet50(pretrained=True)
classifier10=models.resnet50(pretrained=True)'''

classifier_list=[classifier1]
for classifier in classifier_list:
  classifier.to('cuda')

criterion = nn.CrossEntropyLoss()

optimizer_list=[]
for classifier in classifier_list:
    optimizer=optim.Adam(classifier.parameters(), lr=0.001)
    optimizer_list.append(optimizer)



In [ ]:
classifier_clean=models.resnet50(pretrained=True)
optimizer_clean=optim.Adam(classifier_clean.parameters(),lr=0.001)
optimizer_list_clean=[optimizer_clean]

In [ ]:
print(classifier_clean)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#add softmax to the classifier and set the output dimension to be equal to the number of classes
import torch.nn as nn
classifier_clean.fc=nn.Sequential(
    nn.Linear(in_features=2048,out_features=1000,bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1000,out_features=10,bias=True),
    #nn.LogSoftmax(dim=1)
)

In [ ]:
print(classifier_clean)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
classifier_clean=classifier_clean.to('cuda')
classifier_clean_list=[classifier_clean]

In [ ]:
#define the training function
def evaluate_accuracy(model, test_loader):
    # 确保模型在验证模式，这对于某些层如Dropout和BatchNorm是必要的
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():  # 在评估模式下不计算梯度
        for data in test_loader:
            label_tensor=data[1]
            input_tensor=data[0]
            label_tensor=label_tensor.to('cuda')
            input_tensor=input_tensor.to('cuda')
            outputs = model(input_tensor)
            _, predicted = torch.max(outputs.data, 1)
            total += label_tensor.size(0)
            correct += (predicted == label_tensor).sum().item()

    # 转换为百分比
    accuracy = 100 * correct / total
    return accuracy

def train(model_list, train_loader_list, test_loader_list, criterion, optimizer_list, num_epochs=10):
    count_model=0

    new_lst=[]

    for i in range(len(model_list)):
      model=model_list[i]
      model.train()

      train_loader=train_loader_list[i]
      test_loader=test_loader_list[i]
      optimizer=optimizer_list[i]

      for epoch in range(num_epochs):
          running_loss = 0.0
          correct = 0
          total = 0

          # 训练过程
          for data in train_loader:
              #labels, inputs = data[0], data[1]
              label_tensor=data[1]
              input_tensor=data[0]
              #print(label_tensor)
              #print(input_tensor)
              label_tensor=label_tensor.to('cuda')
              input_tensor=input_tensor.to('cuda')
              #print(label_tensor)
              optimizer.zero_grad()  # 清空之前的梯度
              #print(input_tensor.size())
              outputs = model(input_tensor)  # 获得模型输出
              #print(outputs.size())
              #print(outputs.shape)
              #print(label_tensor.shape)
              loss = criterion(outputs, label_tensor)  # 计算损失
              loss.backward()  # 反向传播
              optimizer.step()  # 更新权重

              running_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              #print('this is predicted',predicted)
              total += label_tensor.size(0)
              correct += (predicted == label_tensor).sum().item()

          # 训练完一个epoch后的平均损失
          epoch_loss = running_loss / len(train_loader)

          # 计算测试集上的准确率
          test_accuracy = evaluate_accuracy(model, test_loader)

          print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

          model_path = model_path = '/content/drive/MyDrive/CV/epoch_{}_fine_tuned_resnet50.pth'.format(epoch)
          torch.save(model.state_dict(), model_path)
      count_model+=1

      print('Finished Training',count_model)

      new_lst.append(model)

    return new_lst

In [ ]:
new_model_list=trained_model_list=train(classifier_list, train_loader_lst, test_loader_lst, criterion, optimizer_list, num_epochs=10)

Epoch 1/10, Loss: 1.3775, Test Accuracy: 64.29%
Epoch 2/10, Loss: 1.9195, Test Accuracy: 40.26%
Epoch 3/10, Loss: 1.4930, Test Accuracy: 49.49%
Epoch 4/10, Loss: 1.2868, Test Accuracy: 57.05%
Epoch 5/10, Loss: 1.1120, Test Accuracy: 59.72%
Epoch 6/10, Loss: 0.9604, Test Accuracy: 61.91%
Epoch 7/10, Loss: 0.8273, Test Accuracy: 66.34%
Epoch 8/10, Loss: 0.7031, Test Accuracy: 67.47%
Epoch 9/10, Loss: 0.5826, Test Accuracy: 68.16%
Epoch 10/10, Loss: 0.4657, Test Accuracy: 68.56%
Finished Training 1


In [ ]:
new_clean_model_list=trained_model_list=train(classifier_list, train_loader_full_list, test_loader_full_list, criterion, optimizer_list, num_epochs=10)

Epoch 1/10, Loss: 0.8492, Test Accuracy: 77.97%
Epoch 2/10, Loss: 0.5834, Test Accuracy: 78.65%
Epoch 3/10, Loss: 0.4267, Test Accuracy: 80.97%
Epoch 4/10, Loss: 0.3166, Test Accuracy: 81.04%
Epoch 5/10, Loss: 0.2273, Test Accuracy: 79.94%
Epoch 6/10, Loss: 0.1756, Test Accuracy: 81.06%
Epoch 7/10, Loss: 0.1422, Test Accuracy: 80.01%
Epoch 8/10, Loss: 0.1349, Test Accuracy: 80.30%
Epoch 9/10, Loss: 0.1188, Test Accuracy: 79.75%
Epoch 10/10, Loss: 0.1020, Test Accuracy: 80.21%
Finished Training 1


In [ ]:
#for the result we get bby training on the noisy data is too low, we directly use the model trained on the clean data to
#predict on the test data which was added noise, and check the accuracy
accuracy=evaluate_accuracy(classifier_clean, test_loader_2)
print(accuracy)

10.55


In [ ]:
# #save the state dictionary of thfe fintuned resnet 50 model on the cifar 10 dataset
# model_path = '/content/drive/MyDrive/Classifier_Folder/fine_tuned_resnet50_full.pth'
# torch.save(classifier1.state_dict(), model_path)

In [ ]:
model_path = '/content/drive/MyDrive/CV/fine_tuned_resnet50_noisy_100-200.pth'
torch.save(classifier1.state_dict(), model_path)